## Preparation

In [1]:
import numpy as np
from sympy import *
# from sympy.solvers import solve
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning) 
init_printing(use_unicode=True)

In [2]:
def simulate_M(seed: int, p: int) -> Array:
    np.random.seed(seed=seed)
    normal_matrix = np.random.normal(0, 1, (p, p))
    for i in range(p): # adjust diagonal entries s.t. M stable
        normal_matrix[i, i] = - np.sum(np.abs(normal_matrix[i, :]))
    return normal_matrix

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

def create_A_Sigma(cov: Array, p: int) -> Array:
    row_num = num_sym(p=p)
    col_num = int(p * p)
    A_Sigma = np.empty(shape=(row_num, col_num), dtype=float)
    for l in range(p):
        for k in range(l+1):
            for i in range(p):
                for j in range(p):
                    if (j != k) & (j != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 0
                    elif (j == k) & (k != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[l, i]
                    elif (j == l) & (l != k):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[k, i]
                    elif (j == k) & (j == l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 2 * cov[j, i]
    return A_Sigma

def create_A(A_Sigma: Array, mean: Array, p: int) -> Array:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * eye(p)))
    return np.vstack((A_Sigma, identity))

In [3]:
num_exp = 1000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix

index_intervention = 0
b = 2

### Lasso
$\min_{M\in\R ^{pxp}} \frac{1}{2 n} * \left\lVert c - A * \text{vec}(M) \right\rVert^2_2 + \lambda * \left\lVert \text{vec}(M) \right\rVert_1$

Universally:
* save estimates and their Frobenius distances to the true $M^*$
* $\texttt{frob = -1}$ if no result could be produced

## 2 nodes

In [4]:
p = 2

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]

reg_param = [0.001, 0.01, 0.1, 1]
num_param = len(reg_param) + 1 # also linear regression

### $\texttt{index\_deletion = 0}$
***Results:*** 
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.311}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.716}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.546}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.939}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.715}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.64}$.

In [5]:
'''
index_deletion = 0
offset = 1

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, 4 + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_0[seed-offset, 0] = est_M
    frobnorm_2_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_0[seed-offset, index+1] = est_M
        frobnorm_2_0[seed-offset, index+1] = frob
        

    # Ordinary solution with sympy.solve
    solution = solve(emp_A * vec_M + c, vec_M)

    if len(solution) == 4:
        est_M_np = matrix2numpy(M.subs(solution)).astype(float)
        frob = np.linalg.norm(est_M_np - true_M_np)

        estimate_2_0[seed - offset, 0] = est_M_np.round(3)
        frobnorm_2_0[seed - offset, 0] = frob
    else:
        frobnorm_2_0[seed - offset, 0] = -1
    '''


'\nindex_deletion = 0\noffset = 1\n\nc = np.hstack((vech_C, - b * unit_v))\nc = np.delete(c, (index_deletion))\n\nestimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)\nfrobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)\n\nfor seed in range(offset, 4 + offset):\n    true_M = simulate_M(seed=seed, p=p)\n\n    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)\n    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)\n\n    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = 10)\n    emp_cov = np.cov(obs, rowvar=False)\n    emp_mean = np.mean(obs, axis=0)\n\n    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)\n    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)\n\n    # Linear regression\n    lin_reg = linear_model.LinearRegression()\n    lin_reg.fit(X=emp_A, y=-c)\n    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))\n    frob = np.linalg.norm(est_M - true_M)\n\n    estimate

In [6]:
index_deletion = 0
offset = 1

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_0[seed-offset, 0] = est_M.round(3)
    frobnorm_2_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_0[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_0[seed-offset, index+1] = frob

In [7]:
np.mean(frobnorm_2_0)

In [8]:
np.mean(frobnorm_2_0, axis=0)

array([np.float64(0.8149207435168655), np.float64(0.9651368666292439),
       np.float64(0.9656801934246148), np.float64(1.4977554525252295),
       np.float64(2.6449216219550697)], dtype=object)

### $\texttt{index\_deletion = 1}$
***Results:*** 
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.092}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.119}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.088}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.93}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.773}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.552}$.

In [9]:
index_deletion = 1
offset = 12

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_1[seed-offset, 0] = est_M.round(3)
    frobnorm_2_1[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_1[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_1[seed-offset, index+1] = frob

In [10]:
np.mean(frobnorm_2_1)

In [11]:
np.mean(frobnorm_2_1, axis=0)

array([np.float64(1.109647579854012), np.float64(1.098175096620818),
       np.float64(1.1259596413563264), np.float64(1.5344809765349823),
       np.float64(2.544526165275287)], dtype=object)

### $\texttt{index\_deletion = 2}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.515}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.794}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.721}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.315}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{2.155}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.59}$.

In [12]:
index_deletion = 2
offset = 123

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_2[seed-offset, 0] = est_M.round(3)
    frobnorm_2_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_2[seed-offset, index+1] = frob

In [13]:
np.mean(frobnorm_2_2)

In [14]:
np.mean(frobnorm_2_2, axis=0)

array([np.float64(1.8341394399326245), np.float64(1.8040643901133049),
       np.float64(1.9567571459419715), np.float64(2.1297214900596284),
       np.float64(2.580841638732733)], dtype=object)

### $\texttt{index\_deletion = 3}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.587}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.824}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.93}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.35}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{2.114}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.715}$.

In [15]:
index_deletion = 3
offset = 1234

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_3[seed-offset, 0] = est_M.round(3)
    frobnorm_2_3[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_3[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_3[seed-offset, index+1] = frob

In [16]:
np.mean(frobnorm_2_3)

In [17]:
np.mean(frobnorm_2_3, axis=0)

array([np.float64(1.173212079122521), np.float64(1.0992103591417925),
       np.float64(1.0527467957458996), np.float64(1.807932496222883),
       np.float64(2.7161311930542973)], dtype=object)

### $\texttt{index\_deletion = 4}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{2.007}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.984}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.201}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.017}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.623}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.562}$.

In [18]:
index_deletion = 4
offset = 12345

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_4[seed-offset, 0] = est_M.round(3)
    frobnorm_2_4[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_4[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_4[seed-offset, index+1] = frob

In [19]:
np.mean(frobnorm_2_4)

In [20]:
np.mean(frobnorm_2_4, axis=0)

array([np.float64(0.5254359070100943), np.float64(0.6843382765957513),
       np.float64(0.7738859483887953), np.float64(1.4657694011316678),
       np.float64(2.5603087786860352)], dtype=object)

### No deletion
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{12.004}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{50.428}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{3.422}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.891}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.663}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.615}$.

In [21]:
offset = 12345

c = np.hstack((vech_C, - b * unit_v))

estimate_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2[seed-offset, 0] = est_M.round(3)
    frobnorm_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_2[seed-offset, index+1] = frob

In [22]:
np.mean(frobnorm_2)

In [23]:
np.mean(frobnorm_2, axis=0)

array([np.float64(0.31640154268575704), np.float64(0.25661614152081935),
       np.float64(0.4347665218428023), np.float64(1.4850007438856891),
       np.float64(2.6153284947527413)], dtype=object)

### Overview: 2 nodes
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{3.931}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{10.478}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.652}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{2.074}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.841}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.612}$.

In [24]:
frobnorm_2nodes = np.vstack((frobnorm_2, frobnorm_2_0, frobnorm_2_1, frobnorm_2_2, frobnorm_2_3, frobnorm_2_4))
np.mean(frobnorm_2nodes, axis=0)

array([np.float64(0.9622928820203176), np.float64(0.9845901884369537),
       np.float64(1.051632707783403), np.float64(1.6534434267266764),
       np.float64(2.61034298207602)], dtype=object)

## 3 nodes

In [25]:
p = 3

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
num_param = len(reg_param) + 1 # also linear regression

### All entries
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{42.498}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{271.087}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.678}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.889}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.074}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.614}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.179}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.964}$.

In [26]:
offset = 120

estimate_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_0[seed-offset, 0] = est_M.round(3)
    frobnorm_3_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_0[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_0[seed-offset, index+1] = frob

In [27]:
np.mean(frobnorm_3_0)

In [28]:
np.mean(frobnorm_3_0, axis=0)

array([np.float64(1.5985822382812176), np.float64(1.1979436840885103),
       np.float64(1.4276400870415886), np.float64(1.7503049194905516),
       np.float64(3.251963630348755), np.float64(3.9656645714777334),
       np.float64(4.966101472356572)], dtype=object)

### One random entry missing
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{26.269}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{157.323}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.909}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.935}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.096}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.586}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.118}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.914}$.

In [29]:
entries = [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]]
offset = 1208

estimate_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_1[seed-offset, 0] = est_M.round(3)
    frobnorm_3_1[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_1[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_1[seed-offset, index+1] = frob

In [30]:
np.mean(frobnorm_3_1)

In [31]:
np.mean(frobnorm_3_1, axis=0)

array([np.float64(1.4020310184075546), np.float64(1.1601020306999925),
       np.float64(1.3926530057673174), np.float64(1.7224551100129262),
       np.float64(3.2362609216277045), np.float64(3.9228219251234915),
       np.float64(4.917015244601737)], dtype=object)

### Two entries missing I
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{21.27}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{122.947}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.478}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.765}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.061}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.564}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.129}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.945}$.

In [32]:
offset = 12089

estimate_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_2[seed-offset, 0] = est_M.round(3)
    frobnorm_3_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_2[seed-offset, index+1] = frob

In [33]:
np.mean(frobnorm_3_2)

In [34]:
np.mean(frobnorm_3_2, axis=0)

array([np.float64(1.468500549131823), np.float64(1.1649036826929196),
       np.float64(1.3847339229662154), np.float64(1.7029778835314224),
       np.float64(3.2156604401133935), np.float64(3.9345988575240964),
       np.float64(4.948141349971065)], dtype=object)

### Two entries missing II
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{33.845}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{210.425}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.696}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.849}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.079}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.568}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.117}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.909}$.

In [35]:
offset = 120899

estimate_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_3[seed-offset, 0] = est_M.round(3)
    frobnorm_3_3[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_3[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_3[seed-offset, index+1] = frob

In [36]:
np.mean(frobnorm_3_3)

In [37]:
np.mean(frobnorm_3_3, axis=0)

array([np.float64(1.494461891642084), np.float64(1.1654669389367935),
       np.float64(1.3617718110742767), np.float64(1.6774434061129226),
       np.float64(3.201126109718212), np.float64(3.9004008720783454),
       np.float64(4.91160601172952)], dtype=object)

### Overview: 3 nodes
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{30.97}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{190.445}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{6.758}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{3.86}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.077}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.583}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{4.136}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{4.933}$.

In [38]:
frobnorm_3nodes = np.vstack((frobnorm_3_0, frobnorm_3_1, frobnorm_3_2, frobnorm_3_3))
np.mean(frobnorm_3nodes, axis=0)

array([np.float64(1.4908939243656711), np.float64(1.172104084104554),
       np.float64(1.3916997067123489), np.float64(1.713295329786958),
       np.float64(3.226252775452022), np.float64(3.9308715565509047),
       np.float64(4.9357160196647225)], dtype=object)

## 4 nodes
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{31.527}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{180.115}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{9.611}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{4.943}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{4.77}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{6.478}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{7.185}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{7.589}$.

In [39]:
p = 4

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
num_param = len(reg_param) + 1 # also linear regression
offset = 240

estimate_4 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_4 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_4[seed-offset, 0] = est_M.round(3)
    frobnorm_4[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_4[seed-offset, index+1] = est_M.round(3)
        frobnorm_4[seed-offset, index+1] = frob

np.mean(frobnorm_4)

In [40]:
np.mean(frobnorm_4, axis=0)

array([np.float64(2.4072481489133537), np.float64(2.0861951798924325),
       np.float64(2.7598147910651525), np.float64(3.588047584401723),
       np.float64(6.066892127435979), np.float64(7.131298592786825),
       np.float64(7.589429111215869)], dtype=object)

## 5 nodes
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ is $\large{203.858}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1373.272}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{10.273}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{6.409}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{7.054}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{9.613}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{10.131}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.5$ is $\large{10.253}$.

In [41]:
p = 5

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
num_param = len(reg_param) + 1 # also linear regression
offset = 2409

estimate_5 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_5 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_5[seed-offset, 0] = est_M.round(3)
    frobnorm_5[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_5[seed-offset, index+1] = est_M.round(3)
        frobnorm_5[seed-offset, index+1] = frob

np.mean(frobnorm_5)

In [42]:
np.mean(frobnorm_5, axis=0)

array([np.float64(3.2603465863331853), np.float64(3.0946854496177174),
       np.float64(4.586998208772058), np.float64(5.743267446759196),
       np.float64(9.536728044426047), np.float64(10.1389128267134),
       np.float64(10.253059962608116)], dtype=object)